## Estimating bias in topology-change waiting distances in the MS-SMC'

The solution for the probability that a recombination event *does not change the topology* is exact, however, the expected waiting distance until a topology change event occurs calculated from this probability is not exact, it is approximation. This is because tree-changes (changes to coalescent times but not the topology) can occur during the waiting distance until a topology change. Here we investigate potential biases from this approximation. 


### Approach 1: Analytical solution/approximation
1. Simulate many unlinked genealogies (each for a single site).
2. Calculate `Prob(x-unchanged | S,G)` under MS-SMC' w/ bias for each one.
3. Calculate the `E[waiting distance to x-change]` from each prob.

### Approach 2: Estimation from simulations (slow) 
1. Simulate many unlinked loci of long length and get distance to first x-change event in each.
2. Simulate many unlinked loci of long length and get proportion where the first event is of type x.

In [1]:
from concurrent.futures import ProcessPoolExecutor
from typing import Tuple
import numpy as np
import pandas as pd
import ipcoal
import toyplot, toyplot.svg, toyplot.png
import toytree
from scipy import stats

### Parameters

In [155]:
RECOMB = 2e-9
NSPECIES = 2
NSAMPLES = 4    # samples per species
SPECIES_TREE_HEIGHT = 1e6
NEFF_MIN = 50_000
NEFF_MAX = 500_000
NEFF_VALS = 10
SEED = 123
NLOCI = 4       # very small, increased below after testing
NAME = f"{NSPECIES}-tips"

In [154]:
# get a balanced species tree
sptree = toytree.rtree.baltree(NSPECIES)
sptree = sptree.mod.edges_scale_to_root_height(treeheight=SPECIES_TREE_HEIGHT, include_stem=True)
sptree.draw('p');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="260.0px" viewBox="0 0 350.0 260.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tdb5ebdc66f9b4220865580dfe3b1eb8e"> 0 1 2 r0 r1 0 250000 500000

### Approach 1: get `Prob(x | S,G)` and `E[waiting distance]` under MS-SMC':

This uses the method we describe in the paper as our analytical solution for the probability of a topology change given a species tree, genealogy, and recombination rate. The expected waiting distance is then calculated from this probability. Here we return arrays for the probability of topology-unchanged, and the expected waiting distance until topology change, for many genealogies. The results will be compared with simulated expectations below.

In [9]:
def get_analytical_arrays(sptree: toytree.ToyTree, neff: int, seed: int, **kwargs) -> Tuple:
    """Return Prob(x-change | S,G) and E[waiting-distance] for many genealogies
    where x is either a tree-change or topology-change event.
    
    This uses some global variables (see above).
    
    Parameters
    ----------
    sptree: ToyTree
        A species tree with edge lengths in units of generations
    neff: int
        A diploid effective population size applied to all sptree edges.
    """
    # init a coalescent model using the species tree and diploid Ne value
    model = ipcoal.Model(sptree, Ne=neff, seed_trees=seed, recomb=RECOMB, nsamples=NSAMPLES)
    
    # simulate NLOCI unlinked genealogies
    model.sim_trees(nloci=NLOCI, nsites=1)
    imap = model.get_imap_dict()
    
    # load the first genealogy of every locus as a multitree
    mtree = toytree.mtree(model.df[model.df.tidx == 0].genealogy)
    
    # get Prob(x-change); this func currently takes sptree w/ haploid Ne
    stree = sptree.set_node_data("Ne", default=neff)
    topo_probs = np.array([
        ipcoal.smc.get_probability_of_topology_change(stree, i, imap)
        for i in mtree
    ])
    tree_probs = np.array([
        ipcoal.smc.get_probability_of_tree_change(stree, i, imap)
        for i in mtree
    ])
    
    # get sum branch lengths of each genealogy
    sumlens = np.array([sum(b.dist for b in i if not b.is_root()) for i in mtree])

    # get rate parameters
    topo_rates = sumlens * topo_probs * RECOMB
    tree_rates = sumlens * tree_probs * RECOMB
    
    # get waiting distances
    return (
        tree_probs, 
        topo_probs, 
        np.array([stats.expon.freeze(scale=1 / i).mean() for i in tree_rates]),
        np.array([stats.expon.freeze(scale=1 / i).mean() for i in topo_rates]),
    )

Examples: the probabilities of tree-change or higher, and their waiting-distances lower, than the same calculations for a topology-change event. 

In [10]:
NLOCI = 10

In [11]:
get_analytical_arrays(sptree, neff=50_000, seed=123)

(array([0.46214643, 0.45134642, 0.48727503, 0.45624786, 0.51605831,
        0.46965925, 0.45578032, 0.54121066, 0.43390601, 0.38788795]),
 array([0.11871102, 0.07970322, 0.13171433, 0.16307879, 0.06179511,
        0.10904575, 0.10284207, 0.18694104, 0.10322265, 0.08402183]),
 array([808.43370865, 814.66957807, 685.55839067, 703.5194739 ,
        440.68735759, 789.26058742, 662.00252933, 521.02006381,
        711.54722221, 985.23299735]),
 array([3147.26260687, 4613.34152   , 2536.21221346, 1968.24648182,
        3680.2323288 , 3399.33967474, 2933.89395847, 1508.3986635 ,
        2991.05505424, 4548.34193432]))

### Approach 2: get waiting distances to event x estimated from simulations:

The expected waiting distance for a topology-change under a given MSC model can also be estimated by using simulations. Here we simulate NLOCI tree sequences that each include
at least one recombination event. The first recombination event at each locus can be 
either a no-change, tree-change not affecting topology, or topology-change event. 
For each locus the probability that one of these three events occurs first should be
equal to their relative probability. Therefore, we simply compute the distance until a topology-change event occurs for each locus.

In [111]:
def get_sim_arrays(sptree: toytree.ToyTree, neff: int, nsites: int, seed: int) -> Tuple:
    """Return the proportion of NLOCI simulated tree-sequences that contained a 
    specified event type (tree or topo).
    
    This raises an exception if no events occurred. The nsites arg is used to 
    specify the length of simulated tree-sequences.
    """
    model = ipcoal.Model(
        sptree, Ne=neff, seed_trees=seed, nsamples=NSAMPLES, recomb=RECOMB,
        record_full_arg=True, ancestry_model="smc_prime")
    
    events = np.zeros(NLOCI, dtype=int)
    tree_dist = np.zeros(NLOCI, dtype=int)
    topo_dist = np.zeros(NLOCI, dtype=int)    
    
    # simulate a bunch of long tree-sequences
    for lidx in range(NLOCI):
        # get ts
        ts = next(model._get_tree_sequence_generator(nsites=nsites))

        # get iterators over trees and positions
        trees = ts.trees()
        positions = ts.breakpoints()
        
        # parse first tree
        _ = next(positions)
        t0 = next(trees)
        tlen0 = float(t0.total_branch_length)
        tidx0 = (
            toytree.tree(t0.as_newick())
            .mod.remove_unary_nodes()
            .get_topology_id(exclude_root=False)
        )

        # parse next tree one at a time until tree and topo changes
        this_tree = this_topo = None
        for idx in range(1, 99999999):
            pos = next(positions)
            t1 = next(trees)
            
            # get diff in branch lengths. This will be checked
            # to be > 0.1 to detect a tree-change, to avoid floating
            # point precision errors counting as tree-changes.
            diff = abs(tlen0 - t1.total_branch_length)
            
            # record first event type ----------------------
            if idx == 1:
                if diff < 0.1:
                    events[lidx] = 0
                else:
                    tidx1 = (
                        toytree.tree(t1.as_newick())
                        .mod.remove_unary_nodes()
                        .get_topology_id(exclude_root=False)
                    )
                    if tidx0 == tidx1:
                        events[lidx] = 1
                    else:
                        events[lidx] = 2

            # get distance to tree and topo change --------
            # no-change, check next tree 
            if diff < 0.1:
                continue

            # tree-change
            if not this_tree:
                tree_dist[lidx] = pos
                this_tree = 1

            # topo-change
            if not this_topo:
                tidx1 = (
                    toytree.tree(t1.as_newick())
                    .mod.remove_unary_nodes()
                    .get_topology_id(exclude_root=False)
                )
                if tidx0 != tidx1:
                    topo_dist[lidx] = pos
                    this_topo = 1

            # end loop
            if this_tree and this_topo:
                break

    # count proportion of events of requested type
    tree_prop = events[events != 0].size / NLOCI
    topo_prop = events[events == 2].size / NLOCI
    return tree_prop, topo_prop, tree_dist, topo_dist

In [112]:
NLOCI = 10

In [113]:
get_sim_arrays(sptree, 100_000, 1e6, seed=123)

(0.3,
 0.2,
 array([ 71, 197, 204,  53, 823,  25, 746,  70, 148, 486]),
 array([  71,  197, 1251,   53,  911, 2544,  859,  364,  954,  486]))

## ----

## Compare results of the two methods

In [114]:
# get array to store results of all reps
NLOCI = 100
NREPS = 100
res = np.zeros((NLOCI * NREPS, NEFF_VALS, 8))

# Ne values to test over
nes = np.linspace(NEFF_MIN, NEFF_MAX, NEFF_VALS).astype(int)

In [116]:
# run jobs in parallel to fill array
rasyncs = {}
with ProcessPoolExecutor(max_workers=50) as pool:
    
    # apply a different seed to each rep
    rng = np.random.default_rng(SEED)
    for rep in range(NREPS):
        seed = rng.integers(1e9)
        
        # apply same seed for each diff value of Ne
        for nidx in range(res.shape[1]):

            # get tree-change results
            kwargs = {"sptree": sptree, "neff": nes[nidx], "nsites": int(550000 - nes[nidx]), "seed": seed}
            rasyncs[(nidx, 'smc', rep)] = pool.submit(get_analytical_arrays, **kwargs)
            rasyncs[(nidx, 'sim', rep)] = pool.submit(get_sim_arrays, **kwargs)

# collect results into large res array
for key, future in rasyncs.items():
    nidx, name, rep = key
    ival = slice(NLOCI * rep, NLOCI * (rep + 1))
    if name == 'smc':
        tree_prob, topo_prob, tree_dist, topo_dist = future.result()
        res[ival, nidx, 0] = tree_prob
        res[ival, nidx, 1] = topo_prob
        res[ival, nidx, 2] = tree_dist
        res[ival, nidx, 3] = topo_dist
    else:
        tree_prob, topo_prob, tree_dist, topo_dist = future.result()
        res[ival, nidx, 4] = tree_prob
        res[ival, nidx, 5] = topo_prob
        res[ival, nidx, 6] = tree_dist
        res[ival, nidx, 7] = topo_dist      

In [121]:
# summarize results into a dataframe
data = pd.DataFrame(
    data={
        'tree_smc_prob': res[:, :, 0].mean(axis=0),
        'topo_smc_prob': res[:, :, 1].mean(axis=0),
        'tree_smc_dist': res[:, :, 2].mean(axis=0),
        'topo_smc_dist': res[:, :, 3].mean(axis=0),       
        'tree_sim_prob': res[:, :, 4].mean(axis=0),
        'topo_sim_prob': res[:, :, 5].mean(axis=0),
        'tree_sim_dist': res[:, :, 6].mean(axis=0),
        'topo_sim_dist': res[:, :, 7].mean(axis=0),
        
        'tree_smc_prob_CI95':  tuple(zip(
            np.percentile(res[:, :, 0], 2.5, axis=0).round(5),
            np.percentile(res[:, :, 0], 97.5, axis=0).round(5),
        )),
        'topo_smc_prob_CI95':  tuple(zip(
            np.percentile(res[:, :, 1], 2.5, axis=0).round(5),
            np.percentile(res[:, :, 1], 97.5, axis=0).round(5),
        )),
        
        'tree_smc_dist_CI95':  tuple(zip(
            np.percentile(res[:, :, 2], 2.5, axis=0).round(5),
            np.percentile(res[:, :, 2], 97.5, axis=0).round(5),
        )),
        'topo_smc_dist_CI95':  tuple(zip(
            np.percentile(res[:, :, 3], 2.5, axis=0).round(5),
            np.percentile(res[:, :, 3], 97.5, axis=0).round(5),
        )),
    },
    index=nes,
)
data['nsamples'] = NLOCI * NREPS

In [179]:
# show the dataframe of results
data

,tree_smc_prob,topo_smc_prob,tree_smc_dist,topo_smc_dist,tree_sim_prob,topo_sim_prob,tree_sim_dist,topo_sim_dist,tree_smc_prob_CI95,topo_smc_prob_CI95,tree_smc_dist_CI95,topo_smc_dist_CI95,nsamples
50000,0.476920,0.111093,681.162100,3653.335258,0.4856,0.1150,677.4834,3537.6352,"(0.3541, 0.59016)","(0.034, 0.22744)","(399.06246, 1106.39599)","(1152.89241, 9723.73297)",10000
100000,0.646160,0.224927,364.693777,1315.711438,0.6442,0.2303,357.7984,1268.9563,"(0.53809, 0.72486)","(0.07394, 0.44414)","(220.46783, 579.68244)","(409.93881, 3377.04329)",10000
150000,0.710655,0.314903,262.168948,700.974929,0.7067,0.3081,264.9955,699.3117,"(0.61223, 0.77994)","(0.11148, 0.52123)","(155.82409, 416.81681)","(281.95318, 1775.71688)",10000
200000,0.741575,0.367759,209.292549,476.088313,0.7388,0.3692,210.9816,475.0481,"(0.63912, 0.80972)","(0.14457, 0.5624)","(121.89666, 337.37938)","(220.17439, 1157.30547)",10000
250000,0.759076,0.400453,175.633885,365.599533,0.7581,0.4030,178.9896,363.1672,"(0.64967, 0.82938)","(0.16697, 0.59352)","(100.06938, 288.34939)","(179.23645, 836.41241)",10000
300000,0.770665,0.420918,152.491210,301.622070,0.7739,0.4261,150.7692,295.4857,"(0.65539, 0.84225)","(0.183, 0.61614)","(86.17629, 254.02687)","(152.65501, 642.40944)",10000
350000,0.778340,0.433672,134.938661,258.990523,0.7777,0.4332,136.4608,258.9046,"(0.6591, 0.85185)","(0.1919, 0.6326)","(75.48532, 227.32265)","(134.09949, 524.90023)",10000
400000,0.783735,0.441817,121.253552,228.638948,0.7812,0.4466,121.7994,226.0244,"(0.66032, 0.85917)","(0.19754, 0.64449)","(67.2387, 204.90716)","(119.33003, 447.209)",10000
450000,0.788510,0.448613,110.694135,205.951420,0.7897,0.4566,110.6283,203.6461,"(0.66156, 0.86533)","(0.20095, 0.65497)","(60.56822, 189.0312)","(108.22216, 393.21895)",10000
500000,0.791816,0.452991,101.728303,187.714097,0.7936,0.4566,102.7821,187.2475,"(0.66354, 0.87)","(0.20275, 0.66435)","(55.36008, 175.38847)","(99.11164, 352.16175)",10000


In [156]:
# save the dataframe
data.to_csv(f"./validation-topology-10K-{NAME}.csv")

In [157]:
# save the full array
np.save(f"./validation-topology-10K-{NAME}.npy", res)

### Plot the Prob(topology-unchanged) expectation versus approximation
We expect bias to be greatest at low Ne, where many tree changes happen for every topology-change.

In [258]:
def plot_probs(data, topo: bool=False):
    
    smckey = "topo_smc_" if topo else "tree_smc_"
    simkey = "topo_sim_" if topo else "tree_sim_"
    color = toytree.color.COLORS2[0]
    canvas = toyplot.Canvas(width=350, height=300)
    
    # setup axes
    axb = canvas.cartesian(margin=65)
    axt = axb.share("y")
    axb.x.label.text = "N<sub>e</sub> (diploid)"
    axt.x.label.text = "Sptree edge lengths (coal units)"
    axb.y.label.text = "P(topo-change | S,G)" if topo else "P(tree-change | S,G)"
    axb.y.domain.max = 1
    axb.y.domain.min = 0
    axb.x.domain.min = 0
    
    # style axes
    for ax in (axb.x, axt.x, axb.y):
        ax.domain.show = False
        ax.ticks.show = True
        ax.ticks.near = 7.5
        ax.ticks.far = 0
        ax.ticks.labels.offset = 12
        ax.ticks.labels.style["font-size"] = 16
        ax.label.offset = 35
        ax.label.style["font-size"] = 16
        ax.ticks.style["stroke-width"] = 3
        ax.spine.style["stroke-width"] = 3
        
    avgdist = np.min([i.dist for i in sptree if not i.is_root()])
    axb.x.ticks.locator = toyplot.locator.Explicit(
        np.linspace(50_000, 500_000, 4),
        #np.linspace(0.5, 5, 5),
    )
    axt.x.ticks.locator = toyplot.locator.Explicit(
        np.linspace(50_000, 500_000, 4),
        (avgdist / (2 * np.linspace(50_000, 500_000, 4))).round(1),
    )
        
    # plot data
    axb.hlines([0, 0.5, 1.0], style={"stroke": "grey", "stroke-width": 2, "stroke-dasharray": "6,6", "stroke-opacity": 0.7})
    axb.fill(
       data.index,
       [i[0] for i in data[smckey + "prob_CI95"]],
       [i[1] for i in data[smckey + "prob_CI95"]],
       opacity=0.33,
    )
    axb.plot(data.index, data[smckey + "prob"], stroke_width=2, color=color)
    style = dict(opacity=0.8, color='black', mstyle={"stroke": "none"})
    marks = [
        axb.scatterplot(data.index, data[smckey + "prob"], size=12, color=color, opacity=0.7),
        axb.scatterplot(data.index, data[simkey + "prob"], size=6, marker='s', **style),
    ]
    return canvas

In [261]:
canvas = plot_probs(data, topo=False)
#toyplot.svg.render(canvas, f"../manuscript/figures/alternatives/bias-{NAME}-tree-probs.svg")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="ta708df85680a43618d12c6769cf4dd89"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(tree-change | S,G) 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

In [263]:
canvas = plot_probs(data, topo=True)
#toyplot.svg.render(canvas, f"../manuscript/figures/alternatives/bias-{NAME}-topo-probs.svg")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1c3f3914dc4a4d708963809e8ad24f63"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(topo-change | S,G) 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

In [252]:
def plot_distances(data, topo: bool=False, log: bool=True):
    
    smckey = "topo_smc_" if topo else "tree_smc_"
    simkey = "topo_sim_" if topo else "tree_sim_"
    color = toytree.color.COLORS2[0]
    canvas = toyplot.Canvas(width=350, height=300)
    
    # setup axes
    axb = canvas.cartesian(margin=65)
    axt = axb.share("y")#, yscale="log")
    axb.x.label.text = "N<sub>e</sub> (diploid)"
    axt.x.label.text = "Sptree edge lengths (coal units)"
    axb.y.label.text = "Distance to topo-change" if topo else "Distance to tree-change"
    axb.x.domain.min = 0
        
    if log:
        axt.y.scale = "log"
        axb.y.domain.min = 50
        axb.y.domain.max = 1e4
    else:
        axb.y.domain.min = 10
        axb.y.domain.max = 10000
    
    # style axes
    for ax in (axb.x, axt.x, axb.y):
        ax.domain.show = False
        ax.ticks.show = True
        ax.ticks.near = 7.5
        ax.ticks.far = 0
        ax.ticks.labels.offset = 12
        ax.ticks.labels.style["font-size"] = 16
        ax.label.offset = 35
        ax.label.style["font-size"] = 16
        ax.ticks.style["stroke-width"] = 3
        ax.spine.style["stroke-width"] = 3

    avgdist = np.min([i.dist for i in sptree if not i.is_root()])
    axb.x.ticks.locator = toyplot.locator.Explicit(
        np.linspace(NEFF_MIN, NEFF_MAX, 4),
    )
    axt.x.ticks.locator = toyplot.locator.Explicit(
        np.linspace(NEFF_MIN, NEFF_MAX, 4),
        (avgdist / (2 * np.linspace(NEFF_MIN, NEFF_MAX, 4))).round(1),
    )
    axt.y.ticks.locator = toyplot.locator.Explicit([100, 1000, 10000])

    axb.hlines([100, 1000, 10000], style={"stroke": "grey", "stroke-width": 2, "stroke-dasharray": "6,6", "stroke-opacity": 0.7})

    # plot data
    axb.fill(
       data.index,
       [i[0] for i in data[smckey + "dist_CI95"]],
       [i[1] for i in data[smckey + "dist_CI95"]],
       opacity=0.33,
    )
    axb.plot(data.index, data[smckey + "dist"], stroke_width=3, color=color)
    style = dict(opacity=0.8, color='black', mstyle={"stroke": "none"})
    marks = [
        axb.scatterplot(data.index, data[smckey + "dist"], size=12, marker="o", color=color, opacity=0.7),
        axb.scatterplot(data.index, data[simkey + "dist"], size=6, marker='s', **style),
    ]
    return canvas

In [255]:
canvas = plot_distances(data, topo=False)
#toyplot.svg.render(canvas, f"../manuscript/figures/alternatives/bias-{NAME}-tree-distances.svg")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="teebe23c333e94302bd1bce1aa03bc09c"> <polygon points="87.0,168.35504171093118 109.0,187.39358541949662 131.0,198.52808033506136 153.0,206.4068454531617 175.0,212.7376631790015 197.0,217.53346362893626 219.0,221.7834298556485 241.0,225.49539480568419 263.0,228.8476607864499 285.0,231.7325329030531 285.0,194.73312987668973 263.0,192.3296336982891 241.0,189.74208843400402 219.0,186.41116795461133 197.0,182.8474243309632 175.0,178.7811172400662 153.0,173.74254111707165 131.0,166.95838911505587 109.0,156.3754604644836 87.0,135.63584546347226" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to tree-change 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

In [257]:
canvas = plot_distances(data, topo=True)
#toyplot.svg.render(canvas, f"../manuscript/figures/alternatives/bias-{NAME}-topo-distances.svg")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t8e680ce84c364f7ca736c3ce84fcb5dc"> <polygon points="87.0,134.3150057555007 109.0,167.4922596884568 131.0,179.50085873918488 153.0,187.43631948583632 175.0,194.0367895463856 197.0,199.1873508985171 219.0,203.34560572921765 241.0,207.08964817880565 263.0,210.22463723355415 285.0,213.04622626505468 285.0,172.36662681267978 263.0,168.82834895257267 241.0,164.70022532145404 219.0,159.5606753013561 197.0,153.0788060038034 175.0,144.61153083728817 153.0,134.19242222429358 131.0,120.45623799321513 109.0,99.83169424837217 87.0,65.89889568764637" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to topo-change 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

### Plot observed waiting distance / expected waiting distance 

This is shown for every replicate. In the tree-change scenario the ratio of 

In [172]:
def plot_bias(data, topo: bool=False, nidx: int=0):
    
    smckey = "topo_smc_" if topo else "tree_smc_"
    simkey = "topo_sim_" if topo else "tree_sim_"
    color = toytree.color.COLORS2[0]
    
    canvas = toyplot.Canvas(width=350, height=300)
    
    ax0 = canvas.cartesian(bounds=(65, 250, 65, 300-65))#yscale='log')
    ax1 = canvas.cartesian(bounds=(255, 350-65, 65, 300-65))#yscale='log')
    ax0.label.text = f"Log waiting distance to {'topo' if topo else 'tree'}-change"
    ax0.label.offset = 25
    ax0.x.label.text = "Expected "
    ax0.y.label.text = "Observed/Expected"

    for ax in (ax0.x, ax0.y):
        ax.domain.show = False
        ax.ticks.show = True
        ax.ticks.near = 5
        ax.ticks.far = 0
        ax.ticks.style["stroke-width"] = 2
        ax.ticks.labels.offset = 10
        ax.ticks.labels.style["font-size"] = 14
        ax.label.offset = 35
        ax.label.style["font-size"] = 16
        ax.spine.style["stroke-width"] = 2

    # scatterplot observations
    edata = np.log(res[:, nidx, 3])#[::10]
    odata = np.log(res[:, nidx, 7])#[::10]
    ratio = odata / edata
    print(edata.min(), res[:, nidx, 7].min())
    ax0.scatterplot(edata, ratio, opacity=100 / edata.size, size=6,)
    ax0.hlines(1, style={"stroke-dasharray": "4,4", "stroke-width": 2})
    
    # get histogram of errors
    bins = 25
    mag, pos = np.histogram(ratio, bins=bins)
    ax1.bars(
        pos[:-1] + 0.01, pos[1:] - 0.01, (mag/mag.max()) / 2, 
        along='y', 
        baseline=np.repeat(5.5, bins),
        style={'stroke': "none", "fill": color},
    )
    ax1.show = False
    return canvas
    

In [175]:
np.log(10) / np.log(5), np.log(2)

(1.4306765580733933, 0.6931471805599453)

In [176]:
canvas = plot_bias(data, topo=1, nidx=9)

4.052464195616331 1.0


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t355da8be7162447cb3fab79a7667da86"> <circle r="

In [166]:
toyplot.config.autorender = False
for topo in (0, 1):
    for nidx in range(10):
        canvas = plot_bias(data, topo=topo, nidx=nidx)
        fname = f"bias-{NAME}-{'topo' if topo else 'tree'}-{nidx}.svg"
        toyplot.svg.render(canvas, f"../manuscript/figures/alternatives/{fname}")
toyplot.config.autorender = True

### Stacked histograms